In [ ]:
### Importar librerias

In [37]:
import pandas as pd
import numpy as np
from bdatos import Bdatos
from ingestar import Ingestar

import warnings
warnings.filterwarnings("ignore")

In [38]:
### Verificar estado actual del ingestar y base de datos

print("VERIFICANDO ESTADO ACTUAL DEL SISTEMA")
print("=" * 50)

# Verificar si ya tenemos instancias creadas
try:
    print(f"Ingestar: {type(ingestar)}")
    print("Instancia de Ingestar disponible")
except:
    print("No hay instancia de Ingestar disponible")
    ingestar = Ingestar()
    print("Nueva instancia de Ingestar creada")

# Verificar variables del dataset
print("\nINFORMACIÓN DEL DATASET:")
try:
    print(f"Dataset path: {dataset_path}")
    print(f"Data directory: {data_dir}")
    print(f"Forma del DataFrame: {df_agriculture.shape}")
    print("Dataset cargado correctamente")
    
    # Mostrar información básica del dataset
    print("\nINFORMACIÓN BÁSICA DEL DATASET:")
    print(f"Columnas: {list(df_agriculture.columns)}")
    print(f"Primeras 3 filas:")
    print(df_agriculture.head(3))
    
except Exception as e:
    print(f"Error con el dataset: {e}")

# Verificar estado de la base de datos
print("\nESTADO DE LA BASE DE DATOS:")
try:
    from bdatos import Bdatos
    
    # Si ya existe una instancia de BD
    if 'bd_cultivos' in locals() or 'bd_cultivos' in globals():
        print("Instancia de base de datos encontrada")
        tablas = bd_cultivos.listar_tablas()
        print(f"Tablas existentes: {tablas}")
        
        for tabla in tablas:
            conteo = bd_cultivos.contar_filas(tabla)
            print(f"  - {tabla}: {conteo} filas")
    else:
        print("No se encontró instancia de base de datos")
        
except Exception as e:
    print(f"Error al verificar base de datos: {e}")

print("\n" + "=" * 50)

VERIFICANDO ESTADO ACTUAL DEL SISTEMA
Ingestar: <class 'proyecto_integrador_v.ingestar.Ingestar'>
Instancia de Ingestar disponible

INFORMACIÓN DEL DATASET:
Dataset path: /home/codespace/.cache/kagglehub/datasets/samuelotiattakorah/agriculture-crop-yield/versions/1
Data directory: /home/codespace/.cache/kagglehub/datasets/samuelotiattakorah/agriculture-crop-yield/versions/1
Forma del DataFrame: (1000000, 10)
Dataset cargado correctamente

INFORMACIÓN BÁSICA DEL DATASET:
Columnas: ['Region', 'Soil_Type', 'Crop', 'Rainfall_mm', 'Temperature_Celsius', 'Fertilizer_Used', 'Irrigation_Used', 'Weather_Condition', 'Days_to_Harvest', 'Yield_tons_per_hectare']
Primeras 3 filas:
  Region Soil_Type    Crop  Rainfall_mm  Temperature_Celsius  Fertilizer_Used  \
0   West     Sandy  Cotton   897.077239            27.676966            False   
1  South      Clay    Rice   992.673282            18.026142             True   
2  North      Loam  Barley   147.998025            29.794042            False   

In [40]:
### Probar la nueva clase Bdatos corregida

# Importar la clase Bdatos corregida
try:
    import importlib
    import bdatos
    importlib.reload(bdatos)
    from bdatos import Bdatos
    
    print("Clase Bdatos importada correctamente")
    
    # Crear instancia de la base de datos
    bd_test = Bdatos("static/db/proyecto.db")
    
    # Probar inserción de una muestra del dataset
    df_muestra = df_agriculture.head(1000)  # Tomar solo 1000 filas para prueba
    
    print(f"Insertando muestra de {len(df_muestra)} filas en la base de datos...")
    exito = bd_test.insertar_dataframe(df_muestra, "agricultura_muestra")
    
    if exito:
        # Verificar que se insertó correctamente
        print("Verificando inserción:")
        tablas = bd_test.listar_tablas()
        print(f"Tablas creadas: {tablas}")
        
        for tabla in tablas:
            filas = bd_test.contar_filas(tabla)
            print(f"  - {tabla}: {filas} filas")
        
        # Hacer una consulta de prueba
        print("\nConsulta de prueba - primeras 5 filas:")
        resultado = bd_test.consultar("SELECT * FROM agricultura_muestra LIMIT 5")
        print(resultado)
        
        # Consulta estadística
        print("\nEstadísticas por región:")
        stats = bd_test.consultar("""
            SELECT Region, 
                   COUNT(*) as total_registros,
                   AVG(Yield_tons_per_hectare) as rendimiento_promedio,
                   AVG(Rainfall_mm) as lluvia_promedio
            FROM agricultura_muestra 
            GROUP BY Region
        """)
        print(stats)
    
    bd_test.cerrar_conexion()
    
except Exception as e:
    print(f"Error al probar Bdatos: {e}")
    import traceback
    traceback.print_exc()

Clase Bdatos importada correctamente
 Conectado a la base de datos: static/db/proyecto.db
Insertando muestra de 1000 filas en la base de datos...
 DataFrame insertado en tabla 'agricultura_muestra' (1000 filas)
Verificando inserción:
Tablas creadas: ['agricultura_muestra']
  - agricultura_muestra: 1000 filas

Consulta de prueba - primeras 5 filas:
 Consulta ejecutada exitosamente (5 filas)
  Region Soil_Type     Crop  Rainfall_mm  Temperature_Celsius  \
0   West     Sandy   Cotton   897.077239            27.676966   
1  South      Clay     Rice   992.673282            18.026142   
2  North      Loam   Barley   147.998025            29.794042   
3  North     Sandy  Soybean   986.866331            16.644190   
4  South      Silt    Wheat   730.379174            31.620687   

   Fertilizer_Used  Irrigation_Used Weather_Condition  Days_to_Harvest  \
0                0                1            Cloudy              122   
1                1                1             Rainy              1